In [ ]:
## Here we turn the bucket containing all the images into a RECORDIO file

In [20]:

# An S3 Bucket Name
data_bucket_name='hotel50k-data'

# A prefix name inside the S3 bucket containing sub-folders of images (one per label class)
dataset_name = 'hotel_classes'

In [21]:
!ls

ConvertImages.ipynb
Image-classification-fulltraining-elastic-inference.ipynb
Image-classification-fulltraining-highlevel.ipynb
Image-classification-fulltraining.ipynb
Image-classification-incremental-training-highlevel.ipynb
Image-classification-lst-format-highlevel.ipynb
Image-classification-lst-format.ipynb
Image-classification-transfer-learning-highlevel.ipynb
Image-classification-transfer-learning.ipynb
LoadImages.ipynb


In [22]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()
sess = sagemaker.Session()



In [26]:
base_dir='./tmp'
%env BASE_DIR=$base_dir
%env S3_DATA_BUCKET_NAME = $data_bucket_name
%env DATASET_NAME = $dataset_name

import sys,os

suffix='/mxnet/tools/im2rec.py'
im2rec = list(filter( (lambda x: os.path.isfile(x + suffix )), sys.path))[0] + suffix
%env IM2REC=$im2rec


env: BASE_DIR=./tmp
env: S3_DATA_BUCKET_NAME=hotel50k-data
env: DATASET_NAME=hotel_classes
env: IM2REC=/home/ec2-user/anaconda3/envs/mxnet_latest_p37/cpu/lib/python3.7/site-packages/mxnet/tools/im2rec.py


In [27]:
!pwd
!mkdir $BASE_DIR
!ls

/home/ec2-user/SageMaker/cse281_model
ConvertImages.ipynb
Image-classification-fulltraining-elastic-inference.ipynb
Image-classification-fulltraining-highlevel.ipynb
Image-classification-fulltraining.ipynb
Image-classification-incremental-training-highlevel.ipynb
Image-classification-lst-format-highlevel.ipynb
Image-classification-lst-format.ipynb
Image-classification-transfer-learning-highlevel.ipynb
Image-classification-transfer-learning.ipynb
LoadImages.ipynb
tmp


In [ ]:
!aws s3 sync s3://$S3_DATA_BUCKET_NAME/$DATASET_NAME $BASE_DIR/$DATASET_NAME --quiet

In [13]:
%%bash
# Use the IM2REC script to convert our images into RecordIO files

# Clean up our working dir of existing LST and REC files
cd $BASE_DIR
rm *.rec
rm *.lst

# First we need to create two LST files (training and test lists), noting the correct label class for each image
# We'll also save the output of the LST files command, since it includes a list of all of our label classes
echo "Creating LST files"
python $IM2REC --list --recursive --resize=244 --pass-through --center-crop --test-ratio=0.3 --train-ratio=0.7 $DATASET_NAME $DATASET_NAME > ${DATASET_NAME}_classes

echo "Label classes:"
cat ${DATASET_NAME}_classes

# Then we create RecordIO files from the LST files
echo "Creating RecordIO files"
python $IM2REC --num-thread=4 ${DATASET_NAME}_train.lst $DATASET_NAME
python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME
ls -lh *.rec

Creating LST files
Label classes:
Creating RecordIO files
Creating .rec file from /tmp/hotel_classes_train.lst in /tmp
Creating .rec file from /tmp/hotel_classes_test.lst in /tmp
-rw-rw-r-- 1 ec2-user ec2-user 0 Dec  3 17:27 hotel_classes_test.rec
-rw-rw-r-- 1 ec2-user ec2-user 0 Dec  3 17:27 hotel_classes_train.rec


mkdir: cannot create directory ‘/tmp’: File exists


In [18]:
import pathlib
pathlib.Path().absolute()

PosixPath('/home/ec2-user/SageMaker/cse281_model')

In [16]:
bucket = sess.default_bucket()

s3train_path = 's3://{}/{}/train/'.format(bucket, dataset_name)
s3validation_path = 's3://{}/{}/validation/'.format(bucket, dataset_name)

# Clean up any existing data
!aws s3 rm s3://{bucket}/{dataset_name}/train --recursive
!aws s3 rm s3://{bucket}/{dataset_name}/validation --recursive

# Upload the rec files to the train and validation channels
!aws s3 cp /tmp/{dataset_name}_train.rec $s3train_path
!aws s3 cp /tmp/{dataset_name}_test.rec $s3validation_path

upload: ../../../../tmp/hotel_classes_train.rec to s3://sagemaker-us-east-2-338648799047/hotel_classes/train/hotel_classes_train.rec
upload: ../../../../tmp/hotel_classes_test.rec to s3://sagemaker-us-east-2-338648799047/hotel_classes/validation/hotel_classes_test.rec


In [ ]:
!ls